In [6]:
from fastapi import FastAPI
from pydantic import BaseModel
from python_storybook.core import StoryHub, Story
from typing import List
import os

from core_stories import story_manager
StoryHub.register(story_manager=story_manager)

app = FastAPI()


class GetTitlesReturn(BaseModel):
    titles: List[str]


class GetStoryFullPathsReturn(BaseModel):
    story_full_paths: List[str]


@app.get('/titles', response_model=GetTitlesReturn)
async def get_titles():
    titles = StoryHub.get_titles()
    return GetTitlesReturn(titles=titles)


# @app.get('/changes')

@app.get('/all_story_full_paths', response_model=GetStoryFullPathsReturn)
async def get_all_story_full_paths():
    paths = StoryHub.get_all_story_full_paths()
    return GetStoryFullPathsReturn(story_full_paths=paths)


class GetStoryInput(BaseModel):
    story_full_paths: str


class StoryPatched(Story):
    func: None = None

@app.post('/story', response_model=StoryPatched)
async def get_story(story_path_full_path: GetStoryInput):
    story = StoryHub.get_story(story_path_full_path.story_full_paths)
    story_dump = story.model_dump()
    story_dump['func'] = None
    story = StoryPatched(
        **story_dump
    )
    return story.model_dump()


@app.get('/register_managers', response_model=str)
async def registor_managers():
    StoryHub.register_story_managers(directory=os.getcwd())
    return 'any string'


@app.get('/api-schema')
async def get_api_schema():
    return app.openapi()

## uvicorn main:app --host 0.0.0.0 --port 8080
## uvicorn main:app --host 0.0.0.0 --port 8080 --reload




In [28]:
import json
from fastapi.openapi.utils import get_openapi
from subprocess import run, PIPE
import fastapi
from fastapi import routing as frouting
from starlette import routing as srouting
import fastapi
import starlette

file_names = []
for route in app.routes:

    openapi_schema = get_openapi(
        title="Dynamic Schema Generation",
        version="1.0.0",
        description="Generate schema for specific paths",
        routes=[route]
    )
    base_dir = "schemas/"
    file_name = f"schema_for_{route.path.replace('/', '_')}.json"
    file_names.append(file_name)
    with open(base_dir+file_name, 'w') as f:
        json.dump(openapi_schema, f, indent=4)

In [23]:
def process_routes(file_names, base_dir):

    results = {}

    for file_name in file_names:
        # Run the OpenAPI generator CLI
        path = base_dir+file_name
        
        result = run(
            ['openapi-generator-cli', 'generate', '-i', path, '-g', 'typescript-axios', '-o', f"./ts-client"],
            stdout=PIPE, stderr=PIPE
        )
        # Log result
        results[path] = 'Success' if result.returncode == 0 else result.stderr.decode()

    return results

In [29]:
file_names[0]

'schema_for__openapi.json.json'

In [30]:
results = process_routes(file_names=file_names, base_dir=base_dir)

In [31]:
results

{'schemas/schema_for__openapi.json.json': 'Success',
 'schemas/schema_for__docs.json': 'Success',
 'schemas/schema_for__docs_oauth2-redirect.json': 'Success',
 'schemas/schema_for__redoc.json': 'Success',
 'schemas/schema_for__titles.json': 'Success',
 'schemas/schema_for__all_story_full_paths.json': 'Success',
 'schemas/schema_for__story.json': 'Exception in thread "main" java.util.NoSuchElementException\n\tat java.base/java.util.LinkedHashMap$LinkedHashIterator.nextNode(LinkedHashMap.java:721)\n\tat java.base/java.util.LinkedHashMap$LinkedKeyIterator.next(LinkedHashMap.java:741)\n\tat org.openapitools.codegen.utils.ModelUtils.getType(ModelUtils.java:2141)\n\tat org.openapitools.codegen.utils.ModelUtils.isArraySchema(ModelUtils.java:594)\n\tat org.openapitools.codegen.InlineModelResolver.gatherInlineModels(InlineModelResolver.java:341)\n\tat org.openapitools.codegen.InlineModelResolver.gatherInlineModels(InlineModelResolver.java:296)\n\tat org.openapitools.codegen.InlineModelResolver.

In [3]:
# Additonal
def func(
    # The first number to add
    arg1: int=1,
    # The second number to add
    # Multiline Docstring

    arg2: int=2,
    arg3: int=4, # some noise
):
    '''
    Traditional Docstring
    '''
    # Added number of Arg1 and Arg2
    return arg1 + arg2

In [14]:
import inspect

print(inspect.getsource(func))

def func(
    # The first number to add
    arg1: int=1,
    # The second number to add
    arg2: int=2,
    arg3: int=4,
):
    '''
    Traditional Docstring
    '''
    # Added number of Arg1 and Arg2
    return arg1 + arg2



In [1]:
import ast
import inspect

tree = ast.parse('''def func(
    # The first number to add
    arg1: int=1,
    # The second number to add
    # Multiline Docstring
    arg2: int=2,
    arg3: int=4,
):
    """
    Traditional Docstring
    """
    # Added number of Arg1 and Arg2
    return arg1 + arg2''')

In [24]:
inspect.getsource(func)

"def func(\n    # The first number to add\n    arg1: int=1,\n    # The second number to add\n    # Multiline Docstring\n    arg2: int=2,\n    arg3: int=4,\n):\n    '''\n    Traditional Docstring\n    '''\n    # Added number of Arg1 and Arg2\n    return arg1 + arg2\n"

In [20]:
class A:
    def __init__(self):
        pass
    '''
    aaaa
    '''
    
    @staticmethod
    def func2(self):
        pass
    
    def func3():
        pass

In [21]:
print(inspect.getsource(A.func2))

    @staticmethod
    def func2(self):
        pass



In [24]:
import ast
import tokenize
import io

source_code = inspect.getsource(A.func2)
source_code = '''
@staticmethod
def func2(
    # some docstring
    self
):
    pass
'''


# 토큰화를 사용하여 주석 추출
comments = {}
tokens = tokenize.tokenize(io.BytesIO(source_code.encode('utf-8')).readline)

for token in tokens:
    if token.type == tokenize.COMMENT:
        comments[token.start[0]] = token.string

# AST를 통해 함수의 구조 파악
tree = ast.parse(source_code)
func_details = {}
for node in ast.walk(tree):
    if isinstance(node, ast.FunctionDef):
        func_details['name'] = node.name
        func_details['args'] = {}
        func_details['docstring'] = ast.get_docstring(node)
        for arg in node.args.args:
            arg_comments = []
            # 인자 위의 주석을 모두 포함
            for lineno in range(arg.lineno - 1, 0, -1):
                if lineno in comments:
                    arg_comments.append(comments[lineno])
                else:
                    break
            func_details['args'][arg.arg] = ' '.join(reversed(arg_comments))

        return_node = node.body[-1]
        if isinstance(return_node, ast.Return) and return_node.lineno - 1 in comments:
            func_details['return_comment'] = comments[return_node.lineno - 1]

In [25]:
func_details

{'name': 'func2', 'args': {'self': '# some docstring'}, 'docstring': None}

In [6]:
import ast
import tokenize
import io

source_code = inspect.getsource(func)

# 토큰화를 사용하여 주석 추출
comments = {}
tokens = tokenize.tokenize(io.BytesIO(source_code.encode('utf-8')).readline)

for token in tokens:
    if token.type == tokenize.COMMENT:
        comments[token.start[0]] = token.string

# AST를 통해 함수의 구조 파악
tree = ast.parse(source_code)
func_details = {}
for node in ast.walk(tree):
    if isinstance(node, ast.FunctionDef):
        func_details['name'] = node.name
        func_details['args'] = {}
        func_details['docstring'] = ast.get_docstring(node)
        for arg in node.args.args:
            arg_comments = []
            # 인자 위의 주석을 모두 포함
            for lineno in range(arg.lineno - 1, 0, -1):
                if lineno in comments:
                    arg_comments.append(comments[lineno])
                else:
                    break
            func_details['args'][arg.arg] = ' '.join(reversed(arg_comments))

        return_node = node.body[-1]
        if isinstance(return_node, ast.Return) and return_node.lineno - 1 in comments:
            func_details['return_comment'] = comments[return_node.lineno - 1]



In [7]:
func_details

{'name': 'func',
 'args': {'arg1': '# The first number to add', 'arg2': '', 'arg3': ''},
 'docstring': 'Traditional Docstring',
 'return_comment': '# Added number of Arg1 and Arg2'}